In [ ]:
%tensorflow_version 1.x
import sys
import os

import logging
import multiprocessing
from gensim.models import Word2Vec

import pandas as pd
import sqlite3
import numpy as np
import re
import jieba

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, cross_validate, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB  
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier 
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics

from keras.utils import to_categorical
from tensorflow.python.keras import layers, models, optimizers
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.python.keras.preprocessing import text, sequence
from tensorflow.python.keras.layers import Dense, Input, Flatten, Dropout, LSTM, BatchNormalization
from tensorflow.python.keras import Sequential
from tensorflow.python import keras
from tensorflow.python.keras.callbacks import EarlyStopping

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 這邊記得替換成檔案的路徑
df = pd.read_csv("/content/drive/MyDrive/MosBurger/multi_label_data.csv")

In [ ]:
df.head()

,Unnamed: 0,comment,label,Service,Environment,Food
0,0,日出燒肉蛋堡中間是一顆炸過的水煮蛋味道還不錯,Food,0,0,1
1,1,服務很親切，環境很舒適，餐點很美味，每個人都很殷勤工作，剛剛是“特殊兒”送餐來，很友善認真，...,Service#Enviroment#Food,1,1,1
2,2,店員服務態度佳，常常光顧，不知道為何會有人說店員態度不好，反而是很多客人態度差要求多,Service,1,0,0
3,3,“換”敢不敢寫大一點把雞塊還我= =,Food,0,0,1
4,4,2018-01 平日早上，點塔塔鱈魚堡，感覺size縮水了！服務友善，店內寬敞明亮，有很大的...,Food#Enviroment,0,1,1


In [ ]:
df['comment_seg'] = df['comment'].apply(lambda x: ' '.join(jieba.lcut(x)))
df.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 3.366 seconds.
Prefix dict has been built successfully.


,Unnamed: 0,comment,label,Service,Environment,Food,comment_seg
0,0,日出燒肉蛋堡中間是一顆炸過的水煮蛋味道還不錯,Food,0,0,1,日出 燒 肉蛋 堡 中間 是 一顆 炸過 的 水煮蛋 味道 還不錯
1,1,服務很親切，環境很舒適，餐點很美味，每個人都很殷勤工作，剛剛是“特殊兒”送餐來，很友善認真，...,Service#Enviroment#Food,1,1,1,服務 很 親切 ， 環境 很 舒適 ， 餐點 很 美味 ， 每個 人 都 很 殷勤 工作 ，...
2,2,店員服務態度佳，常常光顧，不知道為何會有人說店員態度不好，反而是很多客人態度差要求多,Service,1,0,0,店員 服務態度 佳 ， 常常 光顧 ， 不 知道 為 何會 有人 說 店員態度 不好 ， 反...
3,3,“換”敢不敢寫大一點把雞塊還我= =,Food,0,0,1,“ 換 ” 敢不敢 寫大 一點 把 雞塊 還我 = =
4,4,2018-01 平日早上，點塔塔鱈魚堡，感覺size縮水了！服務友善，店內寬敞明亮，有很大的...,Food#Enviroment,0,1,1,2018 - 01 平日 早上 ， 點塔塔 鱈 魚堡 ， 感覺 size 縮水 了 ！ ...


In [ ]:
# 將資料中切出 20% 作為測試資料

X_train, X_test, y_train, y_test = train_test_split(df['comment_seg'], df['Environment'], test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2871,) (718,) (2871,) (718,)


In [ ]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', min_df=10)
count_vect.fit(df['comment_seg'])
counts_train = CountVectorizer(vocabulary=count_vect.vocabulary_).fit_transform(X_train)
counts_test = CountVectorizer(vocabulary=count_vect.vocabulary_).fit_transform(X_test)
counts_train

<2871x724 sparse matrix of type '<class 'numpy.int64'>'
	with 16227 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', min_df=10)
tfidf_vect.fit(df['comment_seg'])
tfidf_train = TfidfVectorizer(vocabulary=tfidf_vect.vocabulary_).fit_transform(X_train)
tfidf_test = TfidfVectorizer(vocabulary=tfidf_vect.vocabulary_).fit_transform(X_test)
tfidf_train

<2871x724 sparse matrix of type '<class 'numpy.float64'>'
	with 16227 stored elements in Compressed Sparse Row format>

In [ ]:
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
w2v_model = Word2Vec(df['comment_seg'].apply(lambda x: x.split(' ',-1)), # input要是list不是str
                     min_count=1,
                     size=200,
                     workers=multiprocessing.cpu_count())  # 訓練skip-gram模型

2022-07-13 01:17:24,105 : INFO : running /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-4070a176-5551-49c5-9501-d798d217a044.json
2022-07-13 01:17:24,151 : INFO : collecting all words and their counts
2022-07-13 01:17:24,163 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-07-13 01:17:24,231 : INFO : collected 9238 word types from a corpus of 74369 raw words and 3589 sentences
2022-07-13 01:17:24,238 : INFO : Loading a fresh vocabulary
2022-07-13 01:17:24,271 : INFO : effective_min_count=1 retains 9238 unique words (100% of original 9238, drops 0)
2022-07-13 01:17:24,278 : INFO : effective_min_count=1 leaves 74369 word corpus (100% of original 74369, drops 0)
2022-07-13 01:17:24,328 : INFO : deleting the raw counts dictionary of 9238 items
2022-07-13 01:17:24,336 : INFO : sample=0.001 downsamples 46 most-common words
2022-07-13 01:17:24,342 : INFO : downsampling leaves estimated 53653 word corpus 

In [ ]:
def AvgVector(w2v_model, sentence):
  vec = []
  for i in sentence.split(' ', -1):
    if i in w2v_model.wv.index2word:
      vec.append(w2v_model[i])
  vector = np.mean(vec, axis=0)
  vector = pd.Series(vector)
  return vector

In [ ]:
# %%time
w2v_train = X_train.apply(lambda x: AvgVector(w2v_model, x))
w2v_test = X_test.apply(lambda x: AvgVector(w2v_model, x))
w2v_train.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


(2871, 200)

In [ ]:
# 也可直接讀取詞向量的模型即可
# pretrain_w2v_model = Word2Vec.load("/content/drive/MyDrive/wordvec_wiki/wiki.zh.text.model")

In [ ]:
# %%time
clf = LogisticRegression(max_iter=1000, n_jobs=-1)   
clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(counts_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(counts_test)))
print('='*100)

========================================  Score on Counts feature  ========================================
Score on Train:  0.9212817833507488
Score on Test:  0.8844011142061281


In [ ]:
# %%time
clf = LogisticRegression(max_iter=1000, n_jobs=-1)   
clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(tfidf_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(tfidf_test)))
print('='*100)

========================================  Score on TFIDF feature  ========================================
Score on Train:  0.9056078021595263
Score on Test:  0.871866295264624


In [ ]:
# %%time
clf = LogisticRegression(max_iter=1000, n_jobs=-1)   
clf.fit(w2v_train, y_train)
print('='*40, ' Score on AvgVector feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(w2v_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(w2v_test)))
print('='*100)

========================================  Score on AvgVector feature  ========================================
Score on Train:  0.6283524904214559
Score on Test:  0.6267409470752089


In [ ]:
# %%time
param_grid = {'alpha':[1.4, 1.2, 1, 0.8, 0.6]}
estimators = GridSearchCV(estimator = MultinomialNB(),
                      param_grid = param_grid,
                      n_jobs = -1,
                      cv = 5)
estimators.fit(counts_train, y_train)
print('='*40, ' Score on CV result ', '='*40)
print('Best Score: ', estimators.best_score_)
print('Best Params: ', estimators.best_params_)
clf = MultinomialNB(alpha = estimators.best_params_['alpha'])   
clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(counts_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(counts_test)))
print('='*100)

========================================  Score on CV result  ========================================
Best Score:  0.8599836388425997
Best Params:  {'alpha': 1.4}
========================================  Score on Counts feature  ========================================
Score on Train:  0.8861024033437827
Score on Test:  0.8649025069637883


In [ ]:
# %%time
param_grid = {'alpha':[1.4, 1.2, 1, 0.8, 0.6]}
estimators = GridSearchCV(estimator = MultinomialNB(),
                      param_grid = param_grid,
                      n_jobs = -1,
                      cv = 5)
estimators.fit(tfidf_train, y_train)
print('='*40, ' Score on CV result ', '='*40)
print('Best Score: ', estimators.best_score_)
print('Best Params: ', estimators.best_params_)
# %%time
clf = MultinomialNB(alpha = estimators.best_params_['alpha'])   
clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(tfidf_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(tfidf_test)))
print('='*100)

========================================  Score on CV result  ========================================
Best Score:  0.8728725950613543
Best Params:  {'alpha': 0.6}
========================================  Score on TFIDF feature  ========================================
Score on Train:  0.898989898989899
Score on Test:  0.8649025069637883


In [ ]:
# %%time
scaler = MinMaxScaler()
scaler.fit(w2v_train)

w2v_train_mm = scaler.transform(w2v_train)
w2v_test_mm = scaler.transform(w2v_test)

param_grid = {'alpha':[1.4, 1.2, 1, 0.8, 0.6]}
estimators = GridSearchCV(estimator = MultinomialNB(),
                        param_grid = param_grid,
                        n_jobs = -1,
                        cv = 5)
estimators.fit(w2v_train_mm, y_train)
print('='*40, ' Score on CV result ', '='*40)
print('Best Score: ', estimators.best_score_)
print('Best Params: ', estimators.best_params_)
# %%time
clf = MultinomialNB(alpha = estimators.best_params_['alpha'])   
clf.fit(w2v_train_mm, y_train)
print('='*40, ' Score on AvgVector feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(w2v_train_mm)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(w2v_test_mm)))
print('='*100)

========================================  Score on CV result  ========================================
Best Score:  0.5280424178154824
Best Params:  {'alpha': 1.4}
========================================  Score on AvgVector feature  ========================================
Score on Train:  0.5245559038662487
Score on Test:  0.5125348189415042


In [ ]:
# %%time
clf = SVC(kernel='linear')
clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(counts_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(counts_test)))
print('='*100)

========================================  Score on Counts feature  ========================================
Score on Train:  0.9331243469174504
Score on Test:  0.8941504178272981


In [ ]:
# %%time
clf = SVC(kernel='linear')
clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(tfidf_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(tfidf_test)))
print('='*100)

========================================  Score on TFIDF feature  ========================================
Score on Train:  0.9233716475095786
Score on Test:  0.8885793871866295


In [ ]:
# %%time
clf = SVC(kernel='linear')
clf.fit(w2v_train, y_train)
print('='*40, ' Score on AvgVector feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(w2v_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(w2v_test)))
print('='*100)

========================================  Score on AvgVector feature  ========================================
Score on Train:  0.5977011494252874
Score on Test:  0.5947075208913649


In [ ]:
# %%time
param_grid = {'n_neighbors':list(range(1,9))}
estimators = GridSearchCV(estimator = KNeighborsClassifier(),
                        param_grid = param_grid,
                        n_jobs = -1,
                        cv = 5)
estimators.fit(counts_train, y_train)
print('='*40, ' Score on CV result ', '='*40)
print('Best Score: ', estimators.best_score_)
print('Best Params: ', estimators.best_params_)
# %%time
clf = KNeighborsClassifier(n_neighbors = estimators.best_params_['n_neighbors'])   
clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(counts_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(counts_test)))
print('='*100)

========================================  Score on CV result  ========================================
Best Score:  0.8021639145584002
Best Params:  {'n_neighbors': 3}
========================================  Score on Counts feature  ========================================
Score on Train:  0.8805294322535702
Score on Test:  0.8064066852367688


In [ ]:
# %%time
param_grid = {'n_neighbors':list(range(1,9))}
estimators = GridSearchCV(estimator = KNeighborsClassifier(),
                        param_grid = param_grid,
                        n_jobs = -1,
                        cv = 5)
estimators.fit(tfidf_train, y_train)
print('='*40, ' Score on CV result ', '='*40)
print('Best Score: ', estimators.best_score_)
print('Best Params: ', estimators.best_params_)
# %%time
clf = KNeighborsClassifier(n_neighbors = estimators.best_params_['n_neighbors'], n_jobs=-1)   
clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(tfidf_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(tfidf_test)))
print('='*100)

========================================  Score on CV result  ========================================
Best Score:  0.7662905620360551
Best Params:  {'n_neighbors': 3}
========================================  Score on TFIDF feature  ========================================
Score on Train:  0.9097875304771856
Score on Test:  0.7910863509749304


In [ ]:
# %%time
param_grid = {'n_neighbors':list(range(1,9))}
estimators = GridSearchCV(estimator = KNeighborsClassifier(),
                        param_grid = param_grid,
                        n_jobs = -1,
                        cv = 5)
estimators.fit(w2v_train, y_train)
print('='*40, ' Score on CV result ', '='*40)
print('Best Score: ', estimators.best_score_)
print('Best Params: ', estimators.best_params_)
# %%time
clf = KNeighborsClassifier(n_neighbors = estimators.best_params_['n_neighbors'], n_jobs=-1)   
clf.fit(w2v_train, y_train)
print('='*40, ' Score on AvgVector feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(w2v_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(w2v_test)))
print('='*100)

========================================  Score on CV result  ========================================
Best Score:  0.622428116951977
Best Params:  {'n_neighbors': 6}
========================================  Score on AvgVector feature  ========================================
Score on Train:  0.7290142807384187
Score on Test:  0.6086350974930362


In [ ]:
# %%time
clf = RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', n_jobs=-1, random_state = 10)  
clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(counts_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(counts_test)))
print('='*100)

========================================  Score on Counts feature  ========================================
Score on Train:  0.9780564263322884
Score on Test:  0.8871866295264624


In [ ]:
# %%time
clf = RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', n_jobs=-1, random_state = 10)  
clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(tfidf_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(tfidf_test)))
print('='*100)

========================================  Score on TFIDF feature  ========================================
Score on Train:  0.9780564263322884
Score on Test:  0.8969359331476323


In [ ]:
# %%time
clf = RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', n_jobs=-1, random_state = 10)  
clf.fit(w2v_train, y_train)
print('='*40, ' Score on AvgVector feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(w2v_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(w2v_test)))
print('='*100)

========================================  Score on AvgVector feature  ========================================
Score on Train:  0.9972135144548938
Score on Test:  0.658774373259053


In [ ]:
# %%time
clf = xgb.XGBClassifier(n_estimators=500, objective='multi:softmax', n_jobs=-1, silent=False, num_class=2)
clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(counts_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(counts_test)))
print('='*100)

========================================  Score on Counts feature  ========================================
Score on Train:  0.9261581330546848
Score on Test:  0.8871866295264624


In [ ]:
# %%time
clf = xgb.XGBClassifier(n_estimators=500, objective='multi:softmax', n_jobs=-1, silent=False, num_class=2)
clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(tfidf_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(tfidf_test)))
print('='*100)

========================================  Score on TFIDF feature  ========================================
Score on Train:  0.9373040752351097
Score on Test:  0.8913649025069638


In [ ]:
# %%time
clf = xgb.XGBClassifier(n_estimators=500, objective='multi:softmax', n_jobs=-1, silent=False, num_class=2)
clf.fit(w2v_train, y_train)
print('='*40, ' Score on AvgVector feature ', '='*40)
print('Score on Train: ', metrics.accuracy_score(y_train, clf.predict(w2v_train)))
print('Score on Test: ', metrics.accuracy_score(y_test, clf.predict(w2v_test)))
print('='*100)

========================================  Score on AvgVector feature  ========================================
Score on Train:  0.9752699407871822
Score on Test:  0.6963788300835655


In [ ]:
## 設定超參數
LEARNING_RATE = 1e-3
EPOCHS = 100
BATCH_SIZE = 128
MOMENTUM = 0.95
# 建立資料格式
# 考量時間與記憶體容量，僅保留數量最多的1萬個詞
tokenizer = text.Tokenizer(num_words=10000) 
tokenizer.fit_on_texts(df['comment_seg'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

counts_train = tokenizer.texts_to_sequences(X_train) 
counts_train = tokenizer.sequences_to_matrix(counts_train, mode='freq')
counts_test = tokenizer.texts_to_sequences(X_test)
counts_test = tokenizer.sequences_to_matrix(counts_test, mode='freq')
print('Shape of counts_train tensor:', counts_train.shape)
print('Shape of counts_test tensor:', counts_test.shape)

y_train_dummy = to_categorical(np.asarray(y_train))
y_test_dummy = to_categorical(np.asarray(y_test))
print('Shape of y_train_dummy tensor:', y_train_dummy.shape)
print('Shape of y_test_dummy tensor:', y_test_dummy.shape)
# 搭建模型框架
keras.backend.clear_session()
model = Sequential()
model.add(Dense(units=512, input_shape=(counts_train.shape[1],), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train_dummy.shape[1], activation='softmax'))
model.summary()
# 載入 Callbacks, 並將 monitor 設定為監控 validation loss
earlystop = EarlyStopping(monitor="val_acc", 
                        patience=10, 
                        verbose=1)
model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=LEARNING_RATE, epsilon=None, decay=0.0),
            metrics=['accuracy'])
# %%time
model.fit(counts_train, y_train_dummy,
        epochs=EPOCHS, 
        validation_split=0.2,
        batch_size=BATCH_SIZE,
        shuffle=True,
        callbacks=[earlystop])
model.evaluate(counts_test, y_test_dummy)

Found 9101 unique tokens.
Shape of counts_train tensor: (2871, 10000)
Shape of counts_test tensor: (718, 10000)
Shape of y_train_dummy tensor: (2871, 2)
Shape of y_test_dummy tensor: (718, 2)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-07-13 01:18:46,524 : WARNING : From /tensorflow-1.15.2/python3.7/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               5120512   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 5,288,066
Trainable params: 5,286,530
Non-trainable params: 1,536
__________________________________________

[0.70082599082365, 0.8495822]

In [ ]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 100
BATCH_SIZE = 128
MOMENTUM = 0.95
# 建立資料格式
# 考量時間與記憶體容量，僅保留數量最多的1萬個詞
tokenizer = text.Tokenizer(num_words=10000) 
tokenizer.fit_on_texts(df['comment_seg'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

tfidf_train = tokenizer.texts_to_sequences(X_train) 
tfidf_train = tokenizer.sequences_to_matrix(tfidf_train, mode='tfidf')
tfidf_test = tokenizer.texts_to_sequences(X_test)
tfidf_test = tokenizer.sequences_to_matrix(tfidf_test, mode='tfidf')
print('Shape of tfidf_train tensor:', tfidf_train.shape)
print('Shape of tfidf_test tensor:', tfidf_test.shape)

y_train_dummy = to_categorical(np.asarray(y_train))
y_test_dummy = to_categorical(np.asarray(y_test))
print('Shape of y_train tensor:', y_train_dummy.shape)
print('Shape of y_test tensor:', y_test_dummy.shape)
# 搭建模型框架
keras.backend.clear_session()
model = Sequential()
model.add(Dense(units=512, input_shape=(tfidf_train.shape[1],), activation='relu')) ################
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train_dummy.shape[1], activation='softmax'))
model.summary()
# 載入 Callbacks, 並將 monitor 設定為監控 validation loss
earlystop = EarlyStopping(monitor="val_acc", 
                        patience=5, 
                        verbose=1)
model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=LEARNING_RATE, epsilon=None, decay=0.0),
            metrics=['accuracy'])
# %%time
model.fit(tfidf_train, y_train_dummy,
        epochs=EPOCHS, 
        validation_split=0.2,
        batch_size=BATCH_SIZE,
        shuffle=True,
        callbacks=[earlystop])
model.evaluate(tfidf_test, y_test_dummy)

Found 9101 unique tokens.
Shape of tfidf_train tensor: (2871, 10000)
Shape of tfidf_test tensor: (718, 10000)
Shape of y_train tensor: (2871, 2)
Shape of y_test tensor: (718, 2)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               5120512   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (D

[0.45472575935812715, 0.856546]

In [ ]:
MAX_SEQUENCE_LENGTH = 300 # 每条新闻最大长度
EMBEDDING_DIM = 200 # 词向量空间维度

## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 100
BATCH_SIZE = 128
MOMENTUM = 0.95
# 建立資料格式
tokenizer = text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['comment_seg'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

vec_train = tokenizer.texts_to_sequences(X_train) 
vec_train = sequence.pad_sequences(vec_train, maxlen=MAX_SEQUENCE_LENGTH)
vec_test = tokenizer.texts_to_sequences(X_test)
vec_test = sequence.pad_sequences(vec_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of vec_train tensor:', vec_train.shape)
print('Shape of vec_test tensor:', vec_test.shape)

y_train_dummy = to_categorical(np.asarray(y_train))
y_test_dummy = to_categorical(np.asarray(y_test))
print('Shape of y_train_dummy tensor:', y_train_dummy.shape)
print('Shape of y_test_dummy tensor:', y_test_dummy.shape)
# 將詞替換成對應的向量
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items(): 
    if word in w2v_model:
        embedding_matrix[i] = np.asarray(w2v_model[word],
                                        dtype='float32')
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
# 搭建模型框架
keras.backend.clear_session()
model = Sequential()
model.add(embedding_layer)
model.add(BatchNormalization())

model.add(Conv1D(256, 3, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))

model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(y_train_dummy.shape[1], activation='softmax'))
model.summary()
# 載入 Callbacks, 並將 monitor 設定為監控 validation loss
earlystop = EarlyStopping(monitor="val_acc", 
                        patience=5, 
                        verbose=1)
model.compile(loss='categorical_crossentropy',
            optimizer= optimizers.Adam(lr=LEARNING_RATE, epsilon=None, decay=0.0),
            metrics=['accuracy'])

model.fit(vec_train, y_train_dummy,
        epochs=EPOCHS, 
        validation_split=0.2,
        batch_size=BATCH_SIZE,
        shuffle=True,
        callbacks=[earlystop])
model.evaluate(vec_test, y_test_dummy)

Found 9101 unique tokens.
Shape of vec_train tensor: (2871, 300)
Shape of vec_test tensor: (718, 300)
Shape of y_train_dummy tensor: (2871, 2)
Shape of y_test_dummy tensor: (718, 2)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
2022-07-13 01:19:16,728 : WARNING : From /tensorflow-1.15.2/python3.7/tensorflow_core/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 200)          1820400   
_________________________________________________________________
batch_normalization (BatchNo (None, 300, 200)          800       
_________________________________________________________________
conv1d (Conv1D)              (None, 298, 256)          153856    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 99, 256)           0         
_________________________________________________________________
flatten (Flatten)            (None, 25344)             0         
_________________________________________________________________
dense (Dense)                (None, 200)               5069000   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4

[0.6849386929469521, 0.586351]

In [ ]:
MAX_SEQUENCE_LENGTH = 300 # 每条新闻最大长度
EMBEDDING_DIM = 200 # 词向量空间维度

## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 100
BATCH_SIZE = 128
MOMENTUM = 0.95

# 建立資料格式
tokenizer = text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['comment_seg'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

vec_train = tokenizer.texts_to_sequences(X_train) 
vec_train = sequence.pad_sequences(vec_train, maxlen=MAX_SEQUENCE_LENGTH)
vec_test = tokenizer.texts_to_sequences(X_test)
vec_test = sequence.pad_sequences(vec_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of vec_train tensor:', vec_train.shape)
print('Shape of vec_test tensor:', vec_test.shape)

y_train_dummy = to_categorical(np.asarray(y_train))
y_test_dummy = to_categorical(np.asarray(y_test))
print('Shape of y_train tensor:', y_train_dummy.shape)
print('Shape of y_test tensor:', y_test_dummy.shape)
# 將詞替換成對應的向量
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items(): 
    if word in w2v_model:
        embedding_matrix[i] = np.asarray(w2v_model[word],
                                         dtype='float32')
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
# 搭建模型框架
keras.backend.clear_session()
model = Sequential()
model.add(embedding_layer)
model.add(BatchNormalization())

model.add(Conv1D(256, 3, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))

model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(y_train_dummy.shape[1], activation='softmax'))
model.summary()
# 載入 Callbacks, 並將 monitor 設定為監控 validation loss
earlystop = EarlyStopping(monitor="val_loss", 
                        patience=5, 
                        verbose=1)
model.compile(loss='categorical_crossentropy',
            optimizer= optimizers.Adam(lr=LEARNING_RATE, epsilon=None, decay=0.0),
            metrics=['accuracy'])

model.fit(vec_train, y_train_dummy,
        epochs=EPOCHS, 
        validation_split=0.2,
        batch_size=BATCH_SIZE,
        shuffle=True,
        callbacks=[earlystop])
model.evaluate(vec_test, y_test_dummy)

Found 9101 unique tokens.
Shape of vec_train tensor: (2871, 300)
Shape of vec_test tensor: (718, 300)
Shape of y_train tensor: (2871, 2)
Shape of y_test tensor: (718, 2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 200)          1820400   
_________________________________________________________________
batch_normalization (BatchNo (None, 300, 200)          800       
_________________________________________________________________
conv1d (Conv1D)              (None, 298, 256)          153856    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 99, 256)           0         
_________________________________________________________________
flatten (Flatten)            (None, 25344)             0         
_________________________________________________________________
dense (Dense)                (None, 200)               5069000   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4

[0.6822423312325332, 0.586351]